<a href="https://colab.research.google.com/github/DevNicque/pos-datascience/blob/modulo_4/equipe_4_countries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aquisição, pré-processamento e exploração de dados - Equipe 4

**Alunos**:
* Andrea Monicque dos Santos Silva (amdss.cid20@uea.edu.br)
* Francisco Marcelo Mendes Damasceno (fmmd.cid20@uea.edu.br)
* Marcos Wenneton Araújo (mwvda.cid20@uea.edu.br)
* Lucas Pereira Reis (lpr.cid20@uea.edu.br)


**Atividade**:

Obter um novo Dataset de países a partir de coletar e integrar dados das seguintes fontes:
* https://restcountries.eu/
* Dataset1: flags.csv
* Dataset2: countries.csv

1. Integrar a maior quantidade de informação possível (vocês selecionam qual informação integrar).
2. Criar um novo Dataset e guardar a informação num arquivo .csv

##**Aquisição**

In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path = '/content/drive/Shared drives/Equipe 4 - Aquisição, pré-processamento e exploração de dados/'
df_flags = pd.read_csv(path + 'flags.csv')
df_countries = pd.read_csv(path +'countries.csv', sep=';')

In [9]:
df_flags.head()

,name,landmass,zone,area,population,language,religion,bars,stripes,colours,red,green,blue,gold,white,black,orange,mainhue,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,topleft,botright
0,Afghanistan,5,1,648,16,10,2,0,3,5,1,1,0,1,1,1,0,green,0,0,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,1,0,0,1,0,1,0,red,0,0,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,1,1,0,0,1,0,0,green,0,0,0,0,1,1,0,0,0,0,green,white
3,American Samoa,6,3,0,0,1,1,0,0,5,1,0,1,1,1,0,1,blue,0,0,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,1,0,1,1,0,0,0,gold,0,0,0,0,0,0,0,0,0,0,blue,red


In [10]:
df_countries.head()

,alpha_2,alpha_3,area,capital,continent,currency_code,currency_name,eqivalent_fips_code,fips,geoname_id,languages,name,neighbours,numeric,phone,population,postal_code_format,postal_code_regex,tld
0,AD,AND,468.0,Andorra la Vella,EU,EUR,Euro,NaN,AN,3041565,ca,Andorra,"ES,FR",20,376,84000,AD###,^(?:AD)*(\d{3})$,.ad
1,AE,ARE,82880.0,Abu Dhabi,AS,AED,Dirham,NaN,AE,290557,"ar-AE,fa,en,hi,ur",United Arab Emirates,"SA,OM",784,971,4975593,NaN,NaN,.ae
2,AF,AFG,647500.0,Kabul,AS,AFN,Afghani,NaN,AF,1149361,"fa-AF,ps,uz-AF,tk",Afghanistan,"TM,CN,IR,TJ,PK,UZ",4,93,29121286,NaN,NaN,.af
3,AG,ATG,443.0,St. John's,NaN,XCD,Dollar,NaN,AC,3576396,en-AG,Antigua and Barbuda,NaN,28,+1-268,86754,NaN,NaN,.ag
4,AI,AIA,102.0,The Valley,NaN,XCD,Dollar,NaN,AV,3573511,en-AI,Anguilla,NaN,660,+1-264,13254,NaN,NaN,.ai


##**Pré-Processamento**

In [8]:
#Mudança aplicada antes do merge pra evitar duplicação
df_flags.name = df_flags.name.apply(lambda x: x.replace("-", " "))

In [11]:
# Merge dos dois dataframes
# Caso um país esteja presente nos dois dataframes, será preenchido todas as informações
# Se não, o país é inserido no dataframe com colunas com valor "NaN"

# Aqui desconsiderando as colunas population e area, que em flags tem dados aproximados
merge_df = pd.merge(df_countries, df_flags.drop(columns=['population', 'area']), how="outer", on="name")


merge_df.head()

,alpha_2,alpha_3,area,capital,continent,currency_code,currency_name,eqivalent_fips_code,fips,geoname_id,languages,name,neighbours,numeric,phone,population,postal_code_format,postal_code_regex,tld,landmass,zone,language,religion,bars,stripes,colours,red,green,blue,gold,white,black,orange,mainhue,circles,crosses,saltires,quarters,sunstars,crescent,triangle,icon,animate,text,topleft,botright
0,AD,AND,468.0,Andorra la Vella,EU,EUR,Euro,NaN,AN,3041565.0,ca,Andorra,"ES,FR",20.0,376,84000.0,AD###,^(?:AD)*(\d{3})$,.ad,3.0,1.0,6.0,0.0,3.0,0.0,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,gold,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,blue,red
1,AE,ARE,82880.0,Abu Dhabi,AS,AED,Dirham,NaN,AE,290557.0,"ar-AE,fa,en,hi,ur",United Arab Emirates,"SA,OM",784.0,971,4975593.0,NaN,NaN,.ae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AF,AFG,647500.0,Kabul,AS,AFN,Afghani,NaN,AF,1149361.0,"fa-AF,ps,uz-AF,tk",Afghanistan,"TM,CN,IR,TJ,PK,UZ",4.0,93,29121286.0,NaN,NaN,.af,5.0,1.0,10.0,2.0,0.0,3.0,5.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,green,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,black,green
3,AG,ATG,443.0,St. John's,NaN,XCD,Dollar,NaN,AC,3576396.0,en-AG,Antigua and Barbuda,NaN,28.0,+1-268,86754.0,NaN,NaN,.ag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AI,AIA,102.0,The Valley,NaN,XCD,Dollar,NaN,AV,3573511.0,en-AI,Anguilla,NaN,660.0,+1-264,13254.0,NaN,NaN,.ai,1.0,4.0,1.0,1.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,white,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,white,blue


In [ ]:
# Reduzindo o dataframe para apenas as colunas que são interessantes
# cols_of_interest = ['name', 'language', 'religion', 'area_y', 'population_y', 'capital', 'continent', 'currency_code', 'currency_name', 'phone']

# 1. Aqui, como area e population foram desconsiderados de flags no merge anterior, basta informar area e population sem _y ou _x
# 2. Acrescentadas as colunas de alpha code, importantes mais pra frente
cols_of_interest = ['alpha_2', 'alpha_3', 'name', 'language', 'religion', 'area', 'population', 'capital', 'continent', 'currency_code', 'currency_name', 'phone']


new_df = merge_df[cols_of_interest].copy()

# Renomeando algumas colunas
# new_df.columns = ['name', 'n_languages', 'n_religions', 'area', 'population', 'capital', 'continent', 'currency_code', 'currency_name', 'callingCodes']

# Acrescentando as colunas alpha code
new_df.columns = ['alpha_2', 'alpha_3', 'name', 'n_languages', 'n_religions', 'area', 'population', 'capital', 'continent', 'currency_code', 'currency_name', 'callingCodes']

# Isso aqui foi lá pra cima, o "-" é do dataset flags
# Alguns países estão com "-" para separar os nomes, na API é usado espaço
# new_df.name = new_df.name.apply(lambda x: x.replace("-", " "))

new_df.head()

,alpha_2,alpha_3,name,n_languages,n_religions,area,population,capital,continent,currency_code,currency_name,callingCodes
0,AD,AND,Andorra,6.0,0.0,468.0,84000.0,Andorra la Vella,EU,EUR,Euro,376
1,AE,ARE,United Arab Emirates,NaN,NaN,82880.0,4975593.0,Abu Dhabi,AS,AED,Dirham,971
2,AF,AFG,Afghanistan,10.0,2.0,647500.0,29121286.0,Kabul,AS,AFN,Afghani,93
3,AG,ATG,Antigua and Barbuda,NaN,NaN,443.0,86754.0,St. John's,NaN,XCD,Dollar,+1-268
4,AI,AIA,Anguilla,1.0,1.0,102.0,13254.0,The Valley,NaN,XCD,Dollar,+1-264


In [ ]:
# Exemplo de Resposta da API
response = requests.get('https://restcountries.eu/rest/v2/name/Bonaire')
data = response.json()[0]
print(json.dumps(data, indent=2))

In [ ]:
# Lucas
'''# Criando nova coluna para receber dados da API
new_df['currency_symbol'] = 'default-symbol'

for index, row in new_df.iterrows():
  request_url = "https://restcountries.eu/rest/v2/name/" + row['name']

  response = requests.get(request_url)

  if response.status_code == 200:
    data = response.json()[0]

    new_df.loc[index, 'area'] = data['area']
    new_df.loc[index, 'population'] = data['population']
    new_df.loc[index, 'capital'] = data['capital']
    new_df.loc[index, 'continent'] = data['region']
    new_df.loc[index, 'currency_code'] = data['currencies'][0]['code']
    new_df.loc[index, 'currency_name'] = data['currencies'][0]['name']
    new_df.loc[index, 'currency_symbol'] = data['currencies'][0]['symbol']
    new_df.loc[index, 'callingCodes'] = data['callingCodes'][0]
  
  elif response.status_code == 404:
    print("Not Found: ", row['name'])
  else:
    print("Error: ", response.text)

print("Finished.")'''

In [ ]:
# Como agora são 3 queries diferentes, agrupei o código comum numa função
# pra evitar redundância

def query_restcountries(query_str):
  response = requests.get('https://restcountries.eu/rest/v2/' + query_str)
  # A busca por alphha code quando não encontra nada retorna código 200
  # porém [null], por isso essa checagem do response aqui.
  if response.status_code == 200 and response.json()[0] is not None:
    return response.json()[0]
  elif response.status_code == 404:
    print(f"No results for query: {query_str}")
  else:
    print(f"Error for query: {query_str} - Error message : {response.text}")

# A lógica aqui é tentar a busca por nome, e se não encontrar buscar pelos alpha codes
def get_country(name, alpha2, alpha3):
  result = query_restcountries('name/'+name)
  if result:
    return result
  else:
    if alpha2 != 'nan':
      result = query_restcountries('alpha?codes=' + alpha2)
      if result:
        #print(f"Country {name} Found by alpha code 2: {alpha2}")
        return result
      else:
        if alpha3 != 'nan':
          result = query_restcountries('alpha?codes=' + alpha3)
          if result:
            #print(f"Country {name} Found by alpha code 3: {alpha3}")
            return result

In [ ]:
# Adaptação pra usar a função acima

# Criando nova coluna para receber dados da API
new_df['currency_symbol'] = 'default-symbol'

for index, row in new_df.iterrows():
  data = get_country(row['name'], str(row['alpha_2']), str(row['alpha_3']))
  if data:
    new_df.loc[index, 'area'] = data['area']
    new_df.loc[index, 'population'] = data['population']
    new_df.loc[index, 'capital'] = data['capital']
    new_df.loc[index, 'continent'] = data['region']
    new_df.loc[index, 'currency_code'] = data['currencies'][0]['code']
    new_df.loc[index, 'currency_name'] = data['currencies'][0]['name']
    new_df.loc[index, 'currency_symbol'] = data['currencies'][0]['symbol']
    new_df.loc[index, 'callingCodes'] = data['callingCodes'][0]

print("Finished.")

No results for query: name/Bonaire, Saint Eustatius and Saba 
No results for query: name/Cocos Islands
No results for query: name/Democratic Republic of the Congo
No results for query: name/Republic of the Congo
No results for query: name/Cape Verde
No results for query: name/Czechia
No results for query: name/North Korea
No results for query: name/South Korea
No results for query: name/Palestinian Territory
No results for query: name/British Virgin Islands
No results for query: name/Serbia and Montenegro
Error for query: alpha?codes=CS - Error message : [null]
Error for query: alpha?codes=SCG - Error message : [null]
No results for query: name/Netherlands Antilles
Error for query: alpha?codes=AN - Error message : [null]
Error for query: alpha?codes=ANT - Error message : [null]
No results for query: name/Antigua Barbuda
No results for query: name/British Virgin Isles
No results for query: name/Cape Verde Islands
No results for query: name/Comorro Islands
No results for query: name/Czec

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313 entries, 0 to 312
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             313 non-null    object 
 1   n_languages      194 non-null    float64
 2   n_religions      194 non-null    float64
 3   area             272 non-null    float64
 4   population       283 non-null    float64
 5   capital          282 non-null    object 
 6   continent        280 non-null    object 
 7   currency_code    282 non-null    object 
 8   currency_name    283 non-null    object 
 9   callingCodes     283 non-null    object 
 10  currency_symbol  307 non-null    object 
dtypes: float64(4), object(7)
memory usage: 39.3+ KB


In [ ]:
# Ao final temos menos entradas, acredito que quantidade menor de duplicidade
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291 entries, 0 to 290
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   alpha_2          251 non-null    object 
 1   alpha_3          252 non-null    object 
 2   name             291 non-null    object 
 3   n_languages      194 non-null    float64
 4   n_religions      194 non-null    float64
 5   area             252 non-null    float64
 6   population       263 non-null    float64
 7   capital          263 non-null    object 
 8   continent        262 non-null    object 
 9   currency_code    261 non-null    object 
 10  currency_name    263 non-null    object 
 11  callingCodes     263 non-null    object 
 12  currency_symbol  285 non-null    object 
dtypes: float64(4), object(9)
memory usage: 41.8+ KB


In [ ]:
new_df.head()

,alpha_2,alpha_3,name,n_languages,n_religions,area,population,capital,continent,currency_code,currency_name,callingCodes,currency_symbol
0,AD,AND,Andorra,6.0,0.0,468.0,78014.0,Andorra la Vella,Europe,EUR,Euro,376,€
1,AE,ARE,United Arab Emirates,NaN,NaN,83600.0,9856000.0,Abu Dhabi,Asia,AED,United Arab Emirates dirham,971,د.إ
2,AF,AFG,Afghanistan,10.0,2.0,652230.0,27657145.0,Kabul,Asia,AFN,Afghan afghani,93,؋
3,AG,ATG,Antigua and Barbuda,NaN,NaN,442.0,86295.0,Saint John's,Americas,XCD,East Caribbean dollar,1268,$
4,AI,AIA,Anguilla,1.0,1.0,91.0,13452.0,The Valley,Americas,XCD,East Caribbean dollar,1264,$
